In [0]:
import numpy as np
import torch
import torch.optim as optim

from torch import nn
from torchvision import datasets, transforms

In [2]:
torch.cuda.is_available()

True

In [0]:
class MLP(nn.Module):
  
  def __init__(self, in_feats, out_feats):
    super(MLP, self).__init__()
    
    layer_1 = [
        nn.Linear(in_feats, in_feats*2),
        nn.Sigmoid()
    ]
    
    layer_2 = [
        nn.Linear(in_feats*2, in_feats),
        nn.Sigmoid()
    ]
    
    layer_3 = [
        nn.Linear(in_feats, out_feats),
        nn.Softmax()
    ]
    
    self.nets = nn.Sequential(*layer_1, *layer_2, *layer_3)
    
  def forward(self, x):
    out = self.nets(x)
    return out

In [6]:
x = np.arange(4*3).reshape(3,4)
x = torch.Tensor(x)
net = MLP(4, 1)
net(x)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[1.],
        [1.],
        [1.]], grad_fn=<SoftmaxBackward>)

In [0]:
BATCH_SIZE = 32
EPOCH = 80
LR = 5E-3
MOMENT = 0.9
EVAL_STEP = 2

In [20]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0,), (1.0,))
])

train_set = datasets.MNIST(
    root='', train=True, 
    transform=trans, download=True
)

test_set = datasets.MNIST(
    root='', train=False, 
    transform=trans, download=True
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_set, batch_size=BATCH_SIZE,
    shuffle=False
)

print(len(train_loader), len(test_loader))

1875 313


In [0]:
net = MLP(784, 10)
net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr=LR, momentum=MOMENT)
criterion = nn.CrossEntropyLoss()

In [0]:
def train(net, criterion, optim, loader, epoch):
  net.train()
  total_loss = 0
  for data, label in loader:
    batch_size = data.size(0)
    
    optim.zero_grad()
    data, label = data.cuda(), label.cuda()
    data = data.view(batch_size, -1)
    out = net(data)
    loss = criterion(out, label)
    
    total_loss += loss.item()
    
    loss.backward()
    optim.step()
    
  print("[%03d/%03d] avg loss: %0.4f" %(epoch, EPOCH, total_loss/len(loader)))
  
def test(net, loader):
  net.eval()
  pred = []
  target = []
  for data, label in loader:
    batch_size = data.size(0)
    data = data.view(batch_size, -1)
    data = data.cuda()
    
    pred_batch = net(data)
    pred_batch = pred_batch.to('cpu').detach().numpy()
    pred_batch = np.argmax(pred_batch, 1)
    
    pred.append(pred_batch)
    target.append(label.numpy())
    
    pred = np.concatenate(pred)
    target = np.concatenate(target)
    acc = (np.sum(pred == target) / len(pred))
    
    return pred, target, acc
    

In [29]:
prediction = None
target = None
for epoch in range(1, EPOCH+1):
  train(net, criterion, optimizer, train_loader, epoch)
  if epoch % EVAL_STEP == 0:
    with torch.no_grad():
      prediction, target, acc = test(net, test_loader)
      print(acc)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[001/080] avg loss: 2.2937
[002/080] avg loss: 2.2076
0.40625
[003/080] avg loss: 2.1014
[004/080] avg loss: 2.0080
0.6875
[005/080] avg loss: 1.9277
[006/080] avg loss: 1.8955
0.71875
[007/080] avg loss: 1.8638
[008/080] avg loss: 1.8370
0.6875
[009/080] avg loss: 1.8248
[010/080] avg loss: 1.8163
0.65625
[011/080] avg loss: 1.8092
[012/080] avg loss: 1.8027
0.6875
[013/080] avg loss: 1.7927
[014/080] avg loss: 1.7526
0.8125
[015/080] avg loss: 1.7365
[016/080] avg loss: 1.7290
0.84375
[017/080] avg loss: 1.7240
[018/080] avg loss: 1.7206
0.875
[019/080] avg loss: 1.7178
[020/080] avg loss: 1.7156
0.875
[021/080] avg loss: 1.7139
[022/080] avg loss: 1.7123
0.875
[023/080] avg loss: 1.7109
[024/080] avg loss: 1.7099
0.875
[025/080] avg loss: 1.7088
[026/080] avg loss: 1.7079
0.875
[027/080] avg loss: 1.7070
[028/080] avg loss: 1.7063
0.875
[029/080] avg loss: 1.7055
[030/080] avg loss: 1.7049
0.875
[031/080] avg loss: 1.7042
[032/080] avg loss: 1.7036
0.875
[033/080] avg loss: 1.7032
[